In [1]:
#Import the necessary libraries
import numpy as np
import cv2
import albumentations as A
import torch
from pathlib import Path
import json
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pandas as pd
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import random_split
import neptune
import torch.nn as nn
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold


c:\Users\USER\anaconda3\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
df = pd.read_csv('C:/Users/USER/OneDrive/Masaüstü/MTH221 Project/dataset.csv')
df.head()


,filename,label
0,--FValmNpFJ8yo8X7uWODA_0.jpg,other-sign
1,--FValmNpFJ8yo8X7uWODA_1.jpg,other-sign
2,--FValmNpFJ8yo8X7uWODA_2.jpg,other-sign
3,--L1OMr84-e5nOLbNH5sBA_0.jpg,warning--railroad-crossing-with-barriers--g1
4,--L1OMr84-e5nOLbNH5sBA_1.jpg,other-sign


In [3]:
num_classes = len(df['label'].unique())
num_classes

399

In [5]:
# Custom dataset class for batch loading and transformations
class TrafficSignDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, csv, transformations=None, size=None, flatten=False, label_encoder=None):
        self.image_dir = Path(image_dir)
        self.csv = csv
        self.transformations = transformations
        self.size = size
        self.flatten = flatten
        
        if label_encoder is None:
            self.label_encoder = LabelEncoder()
            self.csv['label'] = self.label_encoder.fit_transform(self.csv['label'])
        else:
            self.label_encoder = label_encoder
            self.csv['label'] = self.label_encoder.transform(self.csv['label'])
            
        # Verify label range
        assert self.csv['label'].max() < num_classes, f"Labels must be < {num_classes}"
        

    def __len__(self):
        return len(self.csv)    
    
    def __getitem__(self, idx):
        # Get image data and load the image
        image_data = self.csv.iloc[idx]
        image = cv2.imread(str(self.image_dir / image_data['filename']))
        
        # Apply transformations (using albumentations)
        if self.transformations: 
            transformed = self.transformations(image=image) 
            image = transformed['image']
        
        # Resize the image
        if self.size is not None:
            image = self.resize_with_pad(image, (self.size, self.size))

        # Flatten the image
        if self.flatten:
            image = image.flatten()
                    
        return image, image_data['label']
    
    def transform(self, image):
        return self.transformations(image)
    

    def resize_with_pad(self,image: np.array, 
                    new_shape: tuple[int, int],  # Fixed: Changed Tuple to tuple
                    padding_color: tuple[int, ...] = (255, 255, 255)) -> np.array:  # Fixed type hint
        
        
        original_shape = (image.shape[1], image.shape[0])
        ratio = float(max(new_shape))/max(original_shape)
        new_size = tuple([int(x*ratio) for x in original_shape])
        image = cv2.resize(image, new_size)
        delta_w = new_shape[0] - new_size[0]
        delta_h = new_shape[1] - new_size[1]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
        return image
    
    # Some helper functions for partial_fit in sklearn
    def get_label_encoded_classes(self):
        classes = self.label_encoder.classes_
        encoded_classes = self.label_encoder.transform(classes)
        return encoded_classes
    

In [6]:
num_classes = len(df['label'].unique())

crops_dir = "crops/"
dataset_csv = "dataset.csv"
df = pd.read_csv(dataset_csv).iloc[:100]

# Hyperparameters
n_epochs = 40
batch_size = 2048
train_size = 0.8
val_size = 0.1
test_size = 0.1
image_size = 256 # 256x256

# Transformations
transformations = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(p=0.3),
    A.Blur(blur_limit=3, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    A.OneOf([
        A.RandomRain(p=1.0),
        A.RandomFog(p=1.0),
        A.RandomSunFlare(p=1.0)
    ], p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the dataset
dataset = TrafficSignDataset(crops_dir, df, size=image_size, flatten=True, transformations=transformations)

# Create the splits
train_dataset, val_dataset, test_dataset = random_split(
    dataset, 
    [train_size, val_size, test_size],
    generator=torch.Generator()
)

# Create data loaders for splits
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

# Define the CNN class
class SimpleCNN(nn.Module):#ChatGPT was used to improve the model
    def __init__(self, num_classes=num_classes, dropout_rate=0.5):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 128x128

            nn.Dropout(dropout_rate),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 64x64

            nn.Dropout(dropout_rate),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 32x32

            nn.Dropout(dropout_rate),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  # 16x16

            nn.Dropout(dropout_rate),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 8x8
        )

        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))

        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(512 * 4 * 4, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = SimpleCNN()

run = neptune.init_run(
    with_id="DEN-1",
    project="ToprakWorkspace/Deneme",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzYTk0ZDYxOS01ZGU1LTRlNTctOGUzNC02OTBiZmQzYjE0NGQifQ==",
    monitoring_namespace="monitoring",
    name = "CNN"
)  

params = {"learning_rate": 0.001, "optimizer": "Adam"}

optimizer = optim.Adam(model.parameters(), lr=0.001)

run["parameters"] = params

for epoch in range(n_epochs):
    # Train
    for batch in train_loader:
        X, y = batch
        X = X.float()
          
        X = X.reshape(-1, 256, 256, 3)
        X = X.permute(0,3,1,2)
        y = y.long()
        y_pred = model(X)
        loss = nn.CrossEntropyLoss()(y_pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        run["train/loss"].log(loss)
        
    # Validation
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            X, y = batch
            X = X.float()
            if len(X.shape) == 2:  
               X = X.reshape(-1, 256, 256, 3)
            X = X.permute(0, 3, 1, 2)
            y = y.long()
            y_pred = model(X)
            all_preds.extend(y_pred.argmax(1).tolist())
            all_labels.extend(y.tolist())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    run["eval/accuracy"].log(accuracy)
    run["eval/precision"].log(precision)
    run["eval/recall"].log(recall)
    run["eval/f1"].log(f1)


# Log the hyperparameters
run["hyperparameters"] = {
    "n_epochs": n_epochs,
    # Classifier Specific Parameters
    "model": {
        "type": "CNN",
        
        "state_dict": model.state_dict(),  # This logs all model parameters
        
    },
    # Dataset Specific Parameters
    "dataset": {
        "batch_size": batch_size,
        "total_size": len(df),
        "train_size": train_size,
        "val_size": val_size,
        "test_size": test_size,
        "n_classes": len(dataset.get_label_encoded_classes()),
        "image_size": image_size,
        "flattened": True,
        "transformations": transformations
    },
}

run.stop()




[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ToprakWorkspace/Deneme/e/DEN-1


KeyboardInterrupt: 

In [7]:

# Function to extract features and labels from the DataLoader
def extract_features_and_labels(loader):
    features = []
    labels = []
    for batch in loader:
        X, y = batch
        features.append(X.view(X.size(0), -1).numpy())  # Flatten the images
        labels.append(y.numpy())
    return np.vstack(features), np.hstack(labels)

# Extract features and labels from the training and test sets
X_train, y_train = extract_features_and_labels(train_loader)
X_test, y_test = extract_features_and_labels(test_loader)

# Initialize the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.6
Classification Report:
               precision    recall  f1-score   support

           4       0.67      0.86      0.75         7
           5       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1

    accuracy                           0.60        10
   macro avg       0.13      0.17      0.15        10
weighted avg       0.47      0.60      0.53        10



c:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\m

In [8]:
gb = DecisionTreeClassifier(
    max_depth=4,              
    min_samples_split=2,      
    min_samples_leaf=1,       
    random_state=42,      
    splitter="best",
    max_features=None,         
    ccp_alpha=0.0007           
)


gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


Accuracy: 0.5
Classification Report:
               precision    recall  f1-score   support

           4       0.71      0.71      0.71         7
           7       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         0

    accuracy                           0.50        10
   macro avg       0.12      0.12      0.12        10
weighted avg       0.50      0.50      0.50        10



c:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\m

In [9]:

xgb = XGBClassifier(
    n_estimators=100,
    learning_rate=0.007,  
    max_depth=3,
    tree_method='approx',  
    enable_categorical=True,  
    missing=0,
    booster='gbtree',
    random_state=42,
    max_delta_step=1,  
    subsample=0.8,  # Use only 80% of data per tree
    colsample_bytree=0.8  # Use only 80% of features per tree
)



In [10]:
# Define base models and final estimator
base_models = [('rf', clf), ('gb', gb)]
stacking_ensemble = StackingClassifier(
    estimators=base_models,
    final_estimator=xgb,  
    cv=5  
)

# Extract features and labels
X_train, y_train = extract_features_and_labels(train_loader)
X_test, y_test = extract_features_and_labels(test_loader)

# Initialize Neptune run
run = neptune.init_run(
    project="ToprakWorkspace/Deneme",
    name="Stacking_Ensemble_Classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzYTk0ZDYxOS01ZGU1LTRlNTctOGUzNC02OTBiZmQzYjE0NGQifQ==",
)

# Define number of epochs and cross-validation folds
n_epochs = 10
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#ChatGPT helped in this part to improve this structure
# Training loop with cross-validation
for epoch in range(n_epochs):
    fold_accuracies = []
    fold_precisions = []
    fold_recalls = []
    fold_f1s = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        # Split data for this fold
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Train the ensemble
        stacking_ensemble.fit(X_fold_train, y_fold_train)
        
        # Make predictions
        y_fold_pred = stacking_ensemble.predict(X_fold_val)
        
        # Calculate metrics
        fold_acc = accuracy_score(y_fold_val, y_fold_pred)
        fold_prec = precision_score(y_fold_val, y_fold_pred, average='macro', zero_division=0)
        fold_rec = recall_score(y_fold_val, y_fold_pred, average='macro', zero_division=0)
        fold_f1 = f1_score(y_fold_val, y_fold_pred, average='macro', zero_division=0)
        
        # Store metrics
        fold_accuracies.append(fold_acc)
        fold_precisions.append(fold_prec)
        fold_recalls.append(fold_rec)
        fold_f1s.append(fold_f1)
        
        # Log fold-specific metrics
        run[f"metrics/fold_{fold}/accuracy"].log(fold_acc, step=epoch)
        run[f"metrics/fold_{fold}/precision"].log(fold_prec, step=epoch)
        run[f"metrics/fold_{fold}/recall"].log(fold_rec, step=epoch)
        run[f"metrics/fold_{fold}/f1"].log(fold_f1, step=epoch)
    
    # Calculate and log average metrics for this epoch
    avg_accuracy = np.mean(fold_accuracies)
    avg_precision = np.mean(fold_precisions)
    avg_recall = np.mean(fold_recalls)
    avg_f1 = np.mean(fold_f1s)
    
    run["metrics/avg_accuracy"].log(avg_accuracy, step=epoch)
    run["metrics/avg_precision"].log(avg_precision, step=epoch)
    run["metrics/avg_recall"].log(avg_recall, step=epoch)
    run["metrics/avg_f1"].log(avg_f1, step=epoch)
    
    print(f"Epoch {epoch + 1}/{n_epochs}")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1: {avg_f1:.4f}")
    print("-" * 50)

# Final evaluation on test set
final_pred = stacking_ensemble.predict(X_test)
final_accuracy = accuracy_score(y_test, final_pred)
final_report = classification_report(y_test, final_pred)

# Log final results
run["metrics/final_accuracy"] = final_accuracy
run["metrics/final_classification_report"] = final_report
run["parameters"] = {
    "model": "StackingClassifier",
    "base_models": ["RandomForest", "DecisionTree"],
    "final_estimator": "XGBoost",
    "n_epochs": n_epochs,
    "n_folds": n_folds
}

run.stop()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ToprakWorkspace/Deneme/e/DEN-44


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1415: RuntimeWarning: Number of classes in training fold (15) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1415: RuntimeWarning: Number of classes in training fold (16) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1415: RuntimeWarning: Number of classes in tra

KeyboardInterrupt: 

-References:

-DataCamp. (n.d.). XGBoost in Python. Retrieved from https://www.datacamp.com/tutorial/xgboost-in-python

-GeeksforGeeks. (n.d.). Building a Convolutional Neural Network using PyTorch. Retrieved from https://www.geeksforgeeks.org/building-a-convolutional-neural-network-using-pytorch/

-GeeksforGeeks. (n.d.). A Comprehensive Guide to Ensemble Learning. Retrieved from https://www.geeksforgeeks.org/a-comprehensive-guide-to-ensemble-learning/

-GeeksforGeeks. (n.d.). ML | XGBoost (Extreme Gradient Boosting). Retrieved from https://www.geeksforgeeks.org/ml-xgboost-extreme-gradient-boosting/

-GeeksforGeeks. (n.d.). Random Forest Regression in Python. Retrieved from https://www.geeksforgeeks.org/random-forest-regression-in-python/

-Mishra, A. (n.d.). Understanding a Simple CNN using PyTorch: Step by Step Guide. Medium. Retrieved from https://medium.com/@abhinav.mishra123/understanding-a-simple-cnn-using-pytorch-step-by-step-guide-f12bdf834a10

-OpenAI. (n.d.). ChatGPT. Retrieved from https://chatgpt.com/